In [6]:
from itertools import permutations

In [7]:
for i,j in list(permutations(range(4), 2)):
    print(i,j)

0 1
0 2
0 3
1 0
1 2
1 3
2 0
2 1
2 3
3 0
3 1
3 2


In [1]:
import torch as th
import torch.nn as nn
from torch.nn.utils import parameters_to_vector

In [13]:
model = nn.Sequential(
        nn.Linear(2, 3),
        nn.ReLU(),
        nn.Linear(3, 1)
    )
l = list(model.parameters())
l

[Parameter containing:
 tensor([[-0.1449, -0.4023],
         [ 0.1169, -0.4974],
         [-0.4563, -0.1452]], requires_grad=True),
 Parameter containing:
 tensor([-0.5411,  0.2825,  0.2986], requires_grad=True),
 Parameter containing:
 tensor([[0.4828, 0.2428, 0.3273]], requires_grad=True),
 Parameter containing:
 tensor([-0.0014], requires_grad=True)]

In [14]:

l_flatten = [th.flatten(p) for p in l]
l_flatten = tuple(l_flatten)
l_concat = th.cat(l_flatten)
l_concat

In [11]:
nn.utils.parameters_to_vector(model.parameters())

tensor([-0.6803,  0.1259,  0.3251,  0.4099, -0.3818, -0.1896, -0.5102,  0.1751,
        -0.3098, -0.2782,  0.0561, -0.2422,  0.4611], grad_fn=<CatBackward0>)